<p style="font-size:11px;"><em><strong>Créditos</strong>: El contenido de este cuaderno ha sido tomado de varias fuentes, pero especialmente de <a href="https://rpubs.com/jimclark/883880">Jim Clark</a>, <a href="https://cran.r-project.org/web/packages/CARBayes/vignettes/CARBayes.pdf">CARBayes</a>, <a href="https://search.r-project.org/CRAN/refmans/CARBayes/html/00Index.html">CRAN</a>, <a href="https://datascienceplus.com/spatial-regression-in-r-part-1-spamm-vs-glmmtmb/">Lionel Hertzog</a>, <a href="https://codingclubuc3m.rbind.io/post/2019-11-05/">Virgilio Gómez Rubio</a>. El compilador se disculpa por cualquier omisión involuntaria y estaría encantado de agregar un reconocimiento.</em></p>


# Modelos de regresión para dependencia espacial tipo CAR

Los modelos CAR con Campos Aleatorios de Markov (MRF) se utilizan ampliamente en estadísticas espaciales para modelar datos observados, así como 
variables latentes y efectos aleatorios que varían espacialmente. Las estructuras CAR se implementan en modelos jerárquicos con efectos aleatorios latentes 
({cite:t}`schmidt2014`). CAR es un tipo de Campo Aleatorio de Markov (MRF) ({cite:t}`besag1991`), que es el enfoque dominante para 
analizar datos espaciales discretos dentro de un marco jerárquico. Un proceso espacial se considera que tiene propiedades de Markov si el estado 
futuro esperado de un parámetro depende únicamente del estado adyacente inmediato, haciendo que el estado futuro sea condicionalmente independiente. 
Extender esto a múltiples efectos aleatorios da como resultado un MRF (\{cite:t}`clifford1990`, {cite:t}`rue2005`). La forma más común de MRF 
representa una conectividad de vecindad de primer orden en términos de contigüidad, donde las observaciones que comparten un límite se consideran vecinas.

Los efectos aleatorios latentes (o simplemente efectos aleatorios espaciales) ofrecen una forma diferente y muy flexible de capturar la dependencia espacial, a menudo representando la variabilidad espacial no observada o no explicada por las covariables fijas en tu modelo.

Los efectos aleatorios latentes pueden ser de dos tipos principales en el contexto espacial:

* Efecto Aleatorio Espacial Estructurado (Structured Spatial Random Effect): Este componente captura la dependencia espacial "suave" o de "vecindad". Modela la correlación entre observaciones cercanas, asumiendo que las ubicaciones más próximas son más similares. 

* Efecto Aleatorio Espacial No Estructurado (Unstructured Spatial Random Effect): Este es un componente aleatorio independiente e idénticamente distribuido (i.i.d.) para cada ubicación. Captura la variabilidad "ruidosa" que no tiene una estructura espacial discernible. Podría pensarse como una variabilidad aleatoria intrínseca a cada observación o ubicación.

La idea es que la combinación de estos efectos aleatorios (junto con los efectos fijos de las covariables) puede explicar completamente la variación en tu variable dependiente, incluyendo la dependencia espacial.

Varios modelos han sido propuestos dentro de esta clase general de estructuras CAR, incluidos los modelos intrínsecos y de convolución 
({cite:t}`besag1991`), así como alternativas como la propuesta por {cite:t}`leroux2000`. El modelo CAR intrínseco (ICAR) es el 
CAR más simple, que asume un efecto aleatorio autorregresivo espacial para abordar las asociaciones entre regiones vecinas ({cite:t}`besag1991`).
La expectativa condicional es igual a la media de los efectos aleatorios en las unidades de mapeo de terreno vecinas, mientras que la varianza 
condicional es inversamente proporcional al número de sus vecinos. Este modelo representa estructuras de correlación espacial fuertes y puede 
no ser adecuado si los datos están débilmente correlacionados.

{cite:t}`besag1991` propuso el modelo de convolución, o modelo Besag-York-Mollié (BYM), que combina dos efectos aleatorios latentes: un efecto 
latente ICAR ($\rho$) y un efecto latente Gaussiano i.i.d. ($\sigma$). Sin embargo, los dos componentes de efectos aleatorios separados no se pueden 
identificar individualmente, y solo se puede identificar su suma.

{cite:t}`leroux2000` propuso utilizar un único efecto aleatorio en su lugar, que incluye un parámetro ($\rho$) para medir el nivel de 
correlación espacial entre las unidades de mapeo de terreno, tomando valores en el intervalo unitario [0-1]. Este modelo es una variación de los 
modelos BYM e ICAR. Al igual que el modelo ICAR, tiene un parámetro de efecto aleatorio espacial para cada observacion. Sin embargo, la distribución 
condicional incorpora características tanto de efectos aleatorios espaciales estructurados como no estructurados (del modelo BYM) en un solo 
parámetro ($\rho$). El modelo de Leroux generaliza el modelo ICAR y el modelo independiente (es decir, un modelo sin ningún efecto aleatorio 
espacial estructurado). Cuando $\rho=1$, se recupera el modelo ICAR; cuando $\rho=0$, se recupera el modelo independiente. Así, el modelo de 
Leroux busca equilibrar estos dos modelos estimando el valor de $\rho$.

## Modelos con la librería CARBayes en R

```r
library(spBayes)
library(maps)
library(RANN)
library(gjam)
library(CARBayes)
library(CARBayesdata)
library(mgcv)
```

```r
#### Set up a square lattice region
m <- 12
xEast  <- 1:m
xNorth <- 1:m
grid   <- expand.grid(xEast, xNorth)
n      <- nrow(grid)
plot( NULL, xlim = c(0, m), ylim = c(0, m), xlab='East', ylab='North' )
abline(v=grid[,1], h=grid[,2])
text(grid[,1] - .5, grid[,2] - .5, 1:n, cex=.8)
```

Se construye una matriz de vecindad (W).


```r
D <- W <- as.matrix(dist(grid))
W[W != 1] <- 0 
```

```r
Q <- 3
x <- matrix( rnorm(Q*n), n, Q )
x[,1] <- 1
x2    <- x[,2]
x3    <- x[,3]
beta  <- matrix( rnorm(Q), Q, 1)
sigma <- .1
```

```r
form <- as.formula(y ~ x2 + x3)

gaussianModel <- S.CARleroux(formula = form, family  = 'gaussian', W = W, 
                             burnin = 20000, n.sample = 100000, thin = 10, verbose = F)
gaussianModel
```


```r
#random effect
fv <- gaussianModel$fitted.values
mf <- min(fv)
cc <- fv - mf
ss <- seq(0, max(cc), length.out=10)
cc <- findInterval(cc, ss)

colM <- colorRampPalette( c("red","orange","blue"))
colm <- colM(10)

symbols(x=grid[,1], y=grid[,2], squares = cc*0+1, bg=colm[cc],
        fg=colm[cc],inches=F, xlab='East', ylab='North')
```

```r
lambda <- exp(x%*%beta + phi[,2] + rnorm(n, 0, sigma))
y <- rpois(n, lambda)

poissonModel <- S.CARbym(formula=form, family="poisson",
                         W=W, burnin=20000, n.sample=100000, thin=10, verbose=F)
poissonModel
```

## Modelos con la librería INLA en R


Para ilustrar cómo se ajustan los modelos espaciales con INLA, se utilizará el conjunto de datos de leucemia de Nueva York. Éste ha sido ampliamente analizado en la literatura (ver, por ejemplo, Waller y Gotway, 2004) y está disponible en el paquete `DClusterm`. El conjunto de datos registra una serie de casos de leucemia en el norte del estado de Nueva York a nivel de distrito censal. Algunas de las variables en el conjunto de datos son:

-   **Casos:** Número de casos de leucemia en el período 1978-1982.
-   **POP8:** Población en 1980.
-   **PCTOWNHOME:** Proporción de personas que son propietarias de su vivienda.
-   **PCTAGE65P:** Proporción de personas de 65 años o más.
-   **AVGIDIST:** Distancia inversa promedio al sitio de tricloroetileno (TCE) más cercano.

Tenga en cuenta que el interés se centra en la exposición al TCE, utilizando AVGIDIST como un indicador de exposición. Las variables PCTOWNHOME y PCTAGE65P actuarán como posibles factores de confusión que deben incluirse en el modelo. Sin embargo, no lo haremos aquí porque queremos probar cómo los efectos espaciales latentes capturan la variación espacial residual.

El conjunto de datos se puede cargar de la siguiente manera:

```r
library(spdep)
library(DClusterm)
data(NY8)
```

Dado que el interés se centra en estudiar el riesgo de leucemia en el norte del estado de Nueva York, primero se calcula el número esperado de casos. Esto se hace calculando la tasa de mortalidad general (total de casos dividido por la población total) y multiplicándola por la población:


```r
rate <- sum(NY8$Cases) / sum(NY8$POP8)
NY8$Expected <- NY8$POP8 * rate
```

Una vez que se obtiene el número esperado de casos, se puede obtener una estimación aproximada del riesgo con la razón de mortalidad estandarizada (SMR), que se calcula como el número de casos observados dividido por el número de casos esperados.


```r
NY8$SMR <- NY8$Cases / NY8$Expected
```

En epidemiología, es importante producir mapas para mostrar la distribución espacial del riesgo relativo. En este ejemplo, nos centraremos en la ciudad de Syracuse para reducir el tiempo de cómputo necesario para producir el mapa. Por lo tanto, creamos un índice con las áreas de la ciudad de Syracuse:


```r
# Subset Syracuse city
syracuse <- which(NY8$AREANAME == "Syracuse city")
```

Un mapa de enfermedades se puede crear simplemente con la función `spplot` (del paquete `sp`):


```r
library(viridis)
spplot(NY8[syracuse, ], "SMR", #at = c(0.6, 0.9801, 1.055, 1.087, 1.125, 13),
   col.regions = rev(magma(16))) #gray.colors(16, 0.9, 0.4))
```

#### Modelo Gaussiano clásico

El primer modelo que consideraremos es un modelo Gausiano sin efectos aleatorios latentes, ya que proporcionará una línea base para comparar con otros modelos. Para ajustar el modelo con INLA, se utiliza la función `inla`:


```r
library(INLA)

```

```r
m1 <- inla(Cases ~ 1 + AVGIDIST,
   data = as.data.frame(NY8),
   family = "Gaussian",verbose=T,
   E = NY8$Expected, control.predictor = list(compute = TRUE),
   control.compute = list(dic = TRUE, waic = TRUE))

summary(m1)
NY8$FIXED.EFF <- m1$summary.fitted[, "mean"]
```

#### Modelo Gaussiano con efectos aleatorios NO estructurados

Se pueden agregar efectos aleatorios latentes al modelo para tener en cuenta la sobredispersión incluyendo efectos aleatorios Gaussianos i.i.d. en el predictor lineal. Para ajustar el modelo con INLA, primero se crea un índice para identificar los efectos aleatorios (ID). Los efectos aleatorios latentes se especifican con la función `f` en INLA:


```r
NY8$ID <- 1:nrow(NY8)
m2 <- inla(Cases ~ 1 + AVGIDIST + f(ID, model = "iid"),
  data = as.data.frame(NY8), family = "Gaussian", 
  E = NY8$Expected,
  control.predictor = list(compute = TRUE),
  control.compute = list(dic = TRUE, waic = TRUE))

summary(m2)
NY8$IID.EFF <- m2$summary.fitted[, "mean"]
```

```r
spplot(NY8[syracuse, ], c("SMR", "FIXED.EFF", "IID.EFF"),
  col.regions = rev(magma(16)))
```

### Modelo Gaussiano con efectos aleatorios estructurados

Tenemos observaciones $y = {y_i}_{i=1}^n$ donde n es el número de áreas. A y se le asigna una distribución multivariante que tiene en cuenta la dependencia espacial. Una forma común de describir la proximidad espacial en datos discretos es mediante una matriz de adyacencia W. El elemento $W_{i,j}$ es distinto de cero si las áreas i y j son vecinas. Por lo general, dos áreas son vecinas si comparten un límite común.

La matriz de adyacencia se puede calcular utilizando la función `poly2nb` en el paquete `spdep`. Esta función considerará dos áreas como vecinas si sus bordes se tocan al menos en un punto (es decir, adyacencia de "Queen"):

```r
NY8.nb <- poly2nb(NY8)
```

Esto devolverá un objeto `nb` con la definición de la estructura del vecindario.

```r
NY8.nb
```

Además, estos objetos creados con la función `nb` se pueden visualizar gráficamente cuando se conoce información adicional sobre las áreas, como la ubicación de su centro.

```r
plot(NY8) 
plot(NY8.nb, coordinates(NY8), add = TRUE, pch = ".", col = "gray")
```

#### Modelo ICAR

El primer ejemplo se basará en la especificación ICAR. Tenga en cuenta que el efecto latente espacial se define usando la función f. Esto requerirá un índice para identificar los efectos aleatorios en cada área, el tipo de modelo y la matriz de adyacencia. Para ello, se utilizará una matriz dispersa.

```r
# Create sparse adjacency matrix
NY8.mat <- as(nb2mat(NY8.nb, style = "B"), "Matrix")
```

```r
# Fit model
m.icar <- inla(Cases ~ 1 +  AVGIDIST + 
    f(ID, model = "besag", graph = NY8.mat),
  data = as.data.frame(NY8), E = NY8$Expected, family ="Gaussian",
  control.predictor = list(compute = TRUE),
  control.compute = list(dic = TRUE, waic = TRUE))

summary(m.icar)
NY8$ICAR <- m.icar$summary.fitted.values[, "mean"]

```

#### Modelo BYM (Besag, York y Mollié)

El modelo de Besag, York y Mollié incluye dos efectos aleatorios latentes: un efecto latente ICAR y un efecto latente Gaussiano i.i.d. No es necesario definir estos dos efectos latentes si se establece el argumento `model` como "bym" cuando se define el efecto aleatorio latente con la función `f`.

```r
m.bym = inla(Cases ~ 1 +  AVGIDIST + 
    f(ID, model = "bym", graph = NY8.mat),
  data = as.data.frame(NY8), E = NY8$Expected, family ="Gaussian",
  control.predictor = list(compute = TRUE),
  control.compute = list(dic = TRUE, waic = TRUE))

summary(m.bym)
NY8$BYM <- m.bym$summary.fitted.values[, "mean"]
```

#### Modelo de Leroux et al.

Este modelo se define utilizando una "mezcla" de matrices (modelo de Leroux et al.) para definir la matriz de precisión del efecto latente. Este modelo se implementa utilizando el efecto latente `generic1`, que utiliza la siguiente matriz de precisión:

$$Σ^{-1} = 1 / τ (I_n - ρ * λ_max * C); ρ ∈ [0,1)$$

En esta ecuación, C es una matriz y λ_max su autovalor máximo.

Para definir el modelo correcto, debemos tomar la matriz C de la siguiente manera:

$$C = I_n - M; M = diag(n_i) - W$$

Entonces, λ_max = 1 y:

$$Σ^{-1} = 1 / τ (I_n - ρ * λ_max * C) = 1 / τ (I_n - ρ * (I_n - M)) = 1 / τ ((1 - ρ)I_n + ρM)$$

Para ajustar el modelo, el primer paso es crear la matriz M.

```r
ICARmatrix <- Diagonal(nrow(NY8.mat), apply(NY8.mat, 1, sum)) - NY8.mat
Cmatrix <- Diagonal(nrow(NY8), 1) -  ICARmatrix
```

El modelo se ajusta como de costumbre con la función `inla`. Tenga en cuenta que la matriz C se pasa a la función `f` usando el argumento `Cmatrix`:

```r
m.ler = inla(Cases ~ 1 +  AVGIDIST +
    f(ID, model = "generic1", Cmatrix = Cmatrix),
  data = as.data.frame(NY8), E = NY8$Expected, family ="Gaussian",
  control.predictor = list(compute = TRUE),
  control.compute = list(dic = TRUE, waic = TRUE))
summary(m.ler)
NY8$LEROUX <- m.ler$summary.fitted.values[, "mean"]

```

## Ejemplo cuencas y subcuencas

```r
aoi = st_read("https://github.com/edieraristizabal/ModeloMultinivel/blob/main/DATA/df_catchments_spatial.gpkg")
aoi <- aoi %>% mutate_at(c('elev_mean','slope_mean','RainfallDaysmean'), ~(scale(.) %>% as.vector))
aoi <- aoi %>% mutate(cuenca_num = case_when(cuenca == 'Atrato' ~ 1, cuenca == 'Cauca' ~ 2, cuenca == 'Magdalena' ~ 3))
aoi_sp <- as_Spatial(aoi)
```

```r
#Spatial matrix
W.nb <- poly2nb(aoi)
W.mat <- nb2mat(W.nb, style="B")
W.list <- nb2listw(W.nb, style="B")
```

```r
#Standard Poisson model in CARBayes
m1_carbayes <- S.glm(formula=lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + offset(log(area)),
                    data=aoi, family="poisson", 
                    burnin=100000, n.sample=300000, thin=100, n.chains=3, n.cores=3)


m1_carbayes
m1_carbayes$modelfit
aoi_sp$carm1 <- m1_carbayes$fitted.values
moran.mc(x=residuals(m1_carbayes), listw=W.list, nsim=1000)
```

```r
#Leroux=proper
cariid <- S.CARleroux(formula=lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + offset(log(area)),
                      data=aoi, family="poisson", W=W.mat,rho=0.5,
                      burnin=100000, n.sample=300000, thin=100, n.chains=3, n.cores=3)
cariid
cariid$modelfit
plot( cariid$samples$rho, bty = 'n' )
summary(cariid$samples)
summary.beta <- summary(cariid$samples$beta, quantiles=c(0.025, 0.975))
beta.mean <- summary.beta$statistics[ ,"Mean"]
beta.ci <- summary.beta$quantiles
beta.results <- cbind(beta.mean, beta.ci)
rownames(beta.results) <- colnames(cariid$X)
round(beta.results, 5)
aoi_sp$cariid <- cariid$fitted.values
aoi_sp$cariid_res <- cariid$residuals
moran.mc(x=residuals(cariid), listw=W.list, nsim=1000)
```

```r
#Leroux=ICAR
caricar <- S.CARleroux(formula=lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + offset(log(area)),
                       data=aoi, family="poisson", W=W.mat,rho=1,
                       burnin=100000, n.sample=300000, thin=100, n.chains=3, n.cores=3)
caricar
caricar$modelfit
summary(caricar$samples)
summary.beta <- summary(caricar$samples$beta, quantiles=c(0.025, 0.975))
beta.mean <- summary.beta$statistics[ ,"Mean"]
beta.ci <- summary.beta$quantiles
beta.results <- cbind(beta.mean, beta.ci)
rownames(beta.results) <- colnames(caricar$X)
round(beta.results, 5)
aoi_sp$caricar <- caricar$fitted.values
aoi_sp$caricar_res <- caricar$residuals
moran.mc(x=residuals(caricar), listw=W.list, nsim=1000)
```

```r
#BYM
carbym <- S.CARbym(formula=lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + offset(log(area)),
                   data=aoi, family="poisson", W=W.mat,
                   burnin=100000, n.sample=300000, thin=100, n.chains=3, n.cores=3)
carbym
carbym$modelfit
plot( carbym$samples$rho, bty = 'n' )
summary(carbym$samples)
summary.beta <- summary(carbym$samples$beta, quantiles=c(0.025, 0.975))
beta.mean <- summary.beta$statistics[ ,"Mean"]
beta.ci <- summary.beta$quantiles
beta.results <- cbind(beta.mean, beta.ci)
rownames(beta.results) <- colnames(carbym$X)
round(beta.results, 5)
aoi$carbym <- carbym$fitted.values
aoi$carbym_res <- carbym$residuals
moran.mc(x=residuals(carbym), listw=W.list, nsim=1000)
```

```r
carleroux <- S.CARleroux(formula=lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + offset(log(area)),
                  data=aoi, family="poisson", W=W.mat,
                  burnin=100000, n.sample=300000, thin=100, n.chains=3, n.cores=3)
carleroux
carleroux$modelfit
plot( carleroux$samples$rho, bty = 'n' )
summary(carleroux$samples)
summary.beta <- summary(carleroux$samples$beta, quantiles=c(0.025, 0.975))
beta.mean <- summary.beta$statistics[ ,"Mean"]
beta.ci <- summary.beta$quantiles
beta.results <- cbind(beta.mean, beta.ci)
rownames(beta.results) <- colnames(carleroux$X)
round(beta.results, 5)
aoi_sp$carleroux <- carleroux$fitted.values
aoi$carleroux_res <- carleroux$residuals
moran.mc(x=residuals(carleroux), listw=W.list, nsim=1000)
```

```r
ggplot() + geom_sf(data=aoi,aes(fill=carbym),color = "black") +
  annotation_scale(location="br",style = "ticks") +
  annotation_north_arrow(location = "tr",which_north = "true", height = unit(1, "cm"), width = unit(1, "cm"),) +
  scale_fill_gradientn(colors=c("white","red"),name = "Landslides") +
  theme_bw() +
  theme(
    panel.grid.major = element_blank(), 
    panel.grid.minor = element_blank(),
    panel.border = element_rect(color = "black",fill = NA,size = 1),
    axis.ticks.length=unit(-0.1, "cm"),
    axis.text.x = element_text(size = 10, margin = unit(c(t = 1, r = 0, b = 0, l = 0), "mm")),
    axis.text.y = element_text(size = 10, margin = unit(c(t = 0, r = 1, b = 0, l = 0), "mm")),
    legend.text = element_text(size=12),
    legend.title.align = 0,
    legend.position = c(0.34,0.9), 
    legend.key.size = unit(0.5, 'cm'),
    legend.justification = "center",
    legend.direction = "horizontal",
    legend.title = element_text(size=14, vjust = .8, hjust = .5))
```

##### con INLA

```r
#Spatial matrix
aoi.nb <- poly2nb(aoi) #Queen matrix
aoi.mat <- as(nb2mat(aoi.nb, style = "B"), "Matrix")
aoi.listw = nb2listw(aoi.nb)
colnames(aoi.mat) <- rownames(aoi.mat) 
mat <- as.matrix(aoi.mat[1:dim(aoi.mat)[1], 1:dim(aoi.mat)[1]])
```

```r
#Graph
nb2INLA("cl_graph",aoi.nb)
am_adj <-paste(getwd(),"G:/My Drive/INVESTIGACION/POSDOC/Figuras3/inla.graph",sep="")
H<-inla.read.graph(filename="G:/My Drive/INVESTIGACION/POSDOC/Figuras3/inla.graph")
image(inla.graph2matrix(H), xlab = "", ylab = "")
```

```r
summary(m1_inla)
aoi$m1_inla <- m1_inla$summary.fitted[1:nrow(aoi), "mean"]
res_pearson_m1 <- (aoi$lands_rec - m1_inla$summary.fitted.values$mean) / sqrt(m1_inla$summary.fitted.values$mean)
aoi$res_pearson_m1 <- res_pearson_m1
moran_m1 <- moran.mc(x = res_pearson_m1, listw = aoi.listw,nsim = 999, alternative = "greater")
```

```r
#basic random intercept (cuenca)
m2_cuenca <- inla(lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + 
           f(cuenca, model = "iid"),
           offset=log(area), data = as.data.frame(aoi), family = "poisson", 
           control.predictor = list(compute = TRUE),
           control.compute = list(dic = TRUE, waic = TRUE))

summary(m2_cuenca)
m2_cuenca$summary.random$cuenca
aoi$m2_cuenca <- m2_cuenca$summary.fitted[1:nrow(aoi), "mean"]
res_pearson_m2 <- (aoi$lands_rec - m2_cuenca$summary.fitted.values$mean) / sqrt(m2_cuenca$summary.fitted.values$mean)
aoi$res_pearson_m2 <- res_pearson_m2
moran_m2 <- moran.mc(x = res_pearson_m2, listw = aoi.listw,nsim = 999, alternative = "greater")
```

```r
# ICAR model
m3_icar <- inla(formula = lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean + 
                  f(cuenca, model = "iid") +
                  f(id, model = "besag", graph = aoi.mat),
                  offset=log(area), data = as.data.frame(aoi), family ="poisson",
                  control.predictor = list(compute = TRUE),
                  control.compute = list(dic = TRUE, waic = TRUE))

summary(m3_icar)
m3_icar$summary.random$cuenca
aoi$ICAR <- m3_icar$summary.fitted[1:nrow(aoi), "mean"]
res_pearson_m3 <- (aoi$lands_rec - m3_icar$summary.fitted.values$mean) / sqrt(m3_icar$summary.fitted.values$mean)
aoi$res_pearson_m3 <- res_pearson_m3
moran_m3 <- moran.mc(x = res_pearson_m3, listw = aoi.listw,nsim = 999, alternative = "greater")
```

```r
# BYM model
m4_bym<-inla(formula = lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean +
           f(cuenca, model = "iid") +
           f(id, model = "bym",graph = aoi.mat), 
           offset=log(area), data = as.data.frame(aoi), family = "poisson",
           control.compute = list(dic = TRUE, waic=T), 
           control.predictor = list(compute = TRUE))

summary(m4_bym)
m4_bym$summary.random$cuenca
aoi$BYM <- m4_bym$summary.fitted[1:nrow(aoi), "mean"]
aoi_sp$BYM <- m4_bym$summary.fitted.values[, "mean"]
res_pearson_m4 <- (aoi$lands_rec - m4_bym$summary.fitted.values$mean) / sqrt(m4_bym$summary.fitted.values$mean)
aoi$res_pearson_m4 <- res_pearson_m4
moran_m4 <- moran.mc(x = res_pearson_m4, listw = aoi.listw,nsim = 999, alternative = "greater")
```

```r
#Leroux et al. model
m5_leroux <- Diagonal(nrow(aoi.mat), apply(aoi.mat, 1, sum)) - aoi.mat
Cmatrix <- Diagonal(nrow(aoi), 1) -  m5_leroux
max(eigen(Cmatrix)$values) #just to check =1

m5_ler = inla(formula = lands_rec ~ 1 + RainfallDaysmean + elev_mean + slope_mean +
                f(cuenca, elev_mean, model = "iid") +
                f(id, model = "generic1", Cmatrix = Cmatrix),
                offset=log(area), data = as.data.frame(aoi), family ="poisson",
                control.predictor = list(compute = TRUE),
                control.compute = list(dic = TRUE, waic = TRUE))

summary(m5_ler)
m5_ler$summary.random$cuenca
aoi$LEROUX <- m5_ler$summary.fitted[1:nrow(aoi), "mean"]
res_pearson_m5 <- (aoi$lands_rec - m5_ler$summary.fitted.values$mean) / sqrt(m5_ler$summary.fitted.values$mean)
aoi$res_pearson_m5 <- res_pearson_m5
moran_m5 <- moran.mc(x = res_pearson_m5, listw = aoi.listw,nsim = 999, alternative = "greater")
```

```r
cuenca_colors <- c("Atrato" = "green", "Cauca" = "red", "Magdalena" = "blue")

p1 <- ggplot() + 
  geom_sf(data = aoi, aes(fill = res_pearson_m1, color = cuenca), size = 0.7) +
  scale_color_manual(values = cuenca_colors, guide = "none") +
  scale_fill_gradient2(
    low = "yellow", mid = "white", high = "purple", midpoint = 0,
    name = "Res. Pearson"
  ) +
  annotation_scale(
    location = "br", style = "ticks",
    text_cex = 1.0  # Tamaño de fuente en escala
  ) +
  annotation_north_arrow(
    location = "tr", which_north = "true",
    height = unit(1, "cm"), width = unit(1, "cm"),
    pad_x = unit(0.1, "cm"), pad_y = unit(0.1, "cm")
  ) +
  theme_bw() +
  theme(
    legend.position = c(0.3, 0.9),
    legend.direction = "horizontal",
    legend.title = element_text(size = 12),
    legend.text = element_text(size = 11),
    legend.key.height = unit(0.4, "cm"),
    legend.key.width = unit(0.5, "cm"),
    axis.text = element_text(size = 11),          # Coordenadas del mapa
    axis.title = element_text(size = 13),         # Si decides añadir títulos
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    plot.title = element_blank()
  )

p2 <- ggplot(aoi, aes(x = cuenca, y = res_pearson_m1, fill = cuenca)) +
  geom_boxplot(notch = TRUE, outlier.shape = 1) +
  scale_fill_manual(values = cuenca_colors) +
  theme_bw() +
  labs(
    y = "Res. Pearson", x = ""  # Etiqueta eje Y
  ) +
  theme(
    legend.position = "none",
    plot.title = element_blank(),
    axis.text.x = element_text(size = 12),         # Etiquetas cuenca
    axis.text.y = element_text(size = 10),         # Ticks eje Y
    axis.title.y = element_text(size = 12),  # Título eje Y
    aspect.ratio = 1.95
  )

combined_plot <- plot_grid(p1, p2, ncol = 2, rel_widths = c(1.2, 0.8))

final_plot <- ggdraw(combined_plot) +
  draw_plot_label(
    label = c("A", "B"),
    x = c(0.1, 0.7), y = c(0.92, 0.92),
    hjust = 0, vjust = 1,
    fontface = "bold", size = 14)

print(final_plot)
```

Para añadir coeficientes aleatorios espaciales (p. ej. para la pendiente) en un modelo CAR con INLA, lo más sencillo es definir un segundo campo latente que modere justamente ese covariable. La idea es que en la fórmula incluyas:

Un campo latente para el intercepto espacial (como ya haces con f(id, model="besag", graph=graph)), y

Un campo latente “pesado” por la covariable cuya pendiente quieres aleatorizar (por ejemplo slope_mean).

```r
# Primero, crea dos índices idénticos (uno para intercepto y otro para pendiente)
aoi$id_int   <- 1:nrow(aoi)
aoi$id_slope <- aoi$id_int

# Ahora la fórmula:
m6_spatial_slope <- inla(
  lands_rec ~
    1
    + RainfallDaysmean
    + elev_mean
    + slope_mean                  # efecto fijo de la pendiente
    + f(cuenca, model="iid")      # efecto aleatorio no espacial por cuenca
    + f(id_int,   model="besag", graph=cl_graph)              # intercepto espacial
    + f(id_slope, slope_mean,      model="besag", graph=cl_graph),  # pendiente espacial
  offset = log(area),
  family = "poisson",
  data = aoi,
  control.predictor = list(compute = TRUE),
  control.compute   = list(dic = TRUE, waic = TRUE)
)
```


Si tu objetivo fuera que la pendiente de slope_mean variara entre cuencas (no dentro de ellas), basta con usar el índice cuenca en lugar de id_slope. Por ejemplo:

```r
m7_basin_slope <- inla(
  lands_rec ~
    1 +
    RainfallDaysmean +
    elev_mean +
    slope_mean +
    f(cuenca,   model="iid") +                         # intercepto por cuenca
    f(cuenca, slope_mean, model="iid"),                # pendiente aleatoria por cuenca
  offset = log(area),
  family = "poisson",
  data = aoi,
  control.predictor = list(compute=TRUE),
  control.compute   = list(dic=TRUE, waic=TRUE)
)
```


Para incluir a la vez

* Intercepto aleatorio por cuenca,

* Pendientes aleatorias por cuenca para cada covariable, y

* Un efecto CAR espacial a nivel de ID (cada polígono)

puedes usar una fórmula como esta en R‑INLA. Supongamos que ya tienes en tu data.frame aoi:

```r
# índice numérico de cuenca (1,2,3)  
aoi$basin_id <- as.integer(aoi$cuenca)  
# índice numérico de polígono (ID)  
aoi$id        <- 1:nrow(aoi)
```


```r
m_complex <- inla(
  lands_rec ~
    1
    + RainfallDaysmean          # efecto fijo
    + elev_mean                 # efecto fijo
    + slope_mean                # efecto fijo

    # 1) Intercepto aleatorio por cuenca
    + f(basin_id, model="iid")

    # 2) Pendientes aleatorias por cuenca
    + f(basin_id, RainfallDaysmean, model="iid")
    + f(basin_id, elev_mean,        model="iid")
    + f(basin_id, slope_mean,       model="iid")

    # 3) Efecto espacial CAR a nivel de polígono
    + f(id, model="besag", graph=cl_graph),

  data = aoi,
  family = "poisson",
  offset = log(area),
  control.predictor = list(compute = TRUE),
  control.compute   = list(dic = TRUE, waic = TRUE)
)
```
